CREATED FACT SELLER PERFORMANCE

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

spark = (
    SparkSession.builder
    .appName("S3Test")
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .config("spark.hadoop.fs.s3a.aws.credentials.provider", "com.amazonaws.auth.DefaultAWSCredentialsProviderChain")
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .getOrCreate()
)



In [2]:
fact_orders = spark.read.parquet(
    "s3a://pedro-datalake-project/gold/fact_orders/"
)

sellers_dim = spark.read.parquet(
    "s3a://pedro-datalake-project/gold/dim_sellers/"
)


In [3]:
fact_orders = fact_orders.drop(
    "seller_city",
    "seller_state",
    "region"
)
fact_orders.show(5)

+--------------------+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+-------------+--------------------+-------------------+-----+-------------+----------+------------+--------------------+----------------------+-----------+-------------+----------+--------------------+
|         customer_id|           seller_id|            order_id|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|order_item_id|          product_id|shipping_limit_date|price|freight_value|total_paid|payment_type|payment_installments|seller_zip_code_prefix|customer_sk|delivery_days|is_delayed|          created_at|
+--------------------+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------

In [4]:
fact_with_seller = (
    fact_orders.alias("f")
    .join(
        sellers_dim.select(
            "seller_id",
            "seller_city",
            "seller_state",
            "region"
        ).alias("d"),
        "seller_id",
        "left"
    )
)
fact_with_seller.show(5)


+--------------------+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+-------------+--------------------+-------------------+-----+-------------+----------+------------+--------------------+----------------------+-----------+-------------+----------+--------------------+---------------+------------+-------+
|           seller_id|         customer_id|            order_id|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|order_item_id|          product_id|shipping_limit_date|price|freight_value|total_paid|payment_type|payment_installments|seller_zip_code_prefix|customer_sk|delivery_days|is_delayed|          created_at|    seller_city|seller_state| region|
+--------------------+--------------------+--------------------+------------+-----------------------

In [5]:
fact_seller_performance = (
    fact_with_seller
    .groupBy("seller_id", "seller_city", "seller_state", "region")
    .agg(
        F.count("order_id").alias("total_orders"),
        F.countDistinct("customer_id").alias("total_customers"),
        F.sum("price").alias("total_revenue"),
        F.sum("freight_value").alias("total_freight"),
        F.avg("delivery_days").alias("avg_delivery_time"),
        F.sum(
            F.when(F.col("is_delayed"), 1).otherwise(0)
        ).alias("late_deliveries"),
        F.avg("price").alias("avg_order_value")
    )
)
fact_seller_performance.show(5)

+--------------------+------------------+------------+-------+------------+---------------+------------------+------------------+-----------------+---------------+------------------+
|           seller_id|       seller_city|seller_state| region|total_orders|total_customers|     total_revenue|     total_freight|avg_delivery_time|late_deliveries|   avg_order_value|
+--------------------+------------------+------------+-------+------------+---------------+------------------+------------------+-----------------+---------------+------------------+
|d66c11a9572221d92...|    rio de janeiro|          RJ|Sudeste|          16|             16| 776.5600000000001|387.82000000000005|            8.625|              0|48.535000000000004|
|53336f97ddbe53e2a...|         sao paulo|          SP|Sudeste|           1|              1|             599.5|             75.33|             19.0|              0|             599.5|
|8ea394aed8138685a...|          sorocaba|          SP|Sudeste|           2|          

In [6]:
fact_seller_performance = (
    fact_seller_performance
    .withColumn(
        "on_time_rate",
        1 - (F.col("late_deliveries") / F.col("total_orders"))
    )
    .withColumn("created_at", F.current_timestamp())
)
fact_seller_performance.printSchema()

root
 |-- seller_id: string (nullable = true)
 |-- seller_city: string (nullable = true)
 |-- seller_state: string (nullable = true)
 |-- region: string (nullable = true)
 |-- total_orders: long (nullable = false)
 |-- total_customers: long (nullable = false)
 |-- total_revenue: double (nullable = true)
 |-- total_freight: double (nullable = true)
 |-- avg_delivery_time: double (nullable = true)
 |-- late_deliveries: long (nullable = true)
 |-- avg_order_value: double (nullable = true)
 |-- on_time_rate: double (nullable = true)
 |-- created_at: timestamp (nullable = false)



In [7]:
fact_seller_performance.show(10, truncate=False)
fact_seller_performance.count()


+--------------------------------+------------------+------------+-------+------------+---------------+------------------+------------------+------------------+---------------+------------------+------------------+--------------------------+
|seller_id                       |seller_city       |seller_state|region |total_orders|total_customers|total_revenue     |total_freight     |avg_delivery_time |late_deliveries|avg_order_value   |on_time_rate      |created_at                |
+--------------------------------+------------------+------------+-------+------------+---------------+------------------+------------------+------------------+---------------+------------------+------------------+--------------------------+
|d66c11a9572221d92fbb8c4897db5f9b|rio de janeiro    |RJ          |Sudeste|16          |16             |776.5600000000001 |387.82000000000005|8.625             |0              |48.535000000000004|1.0               |2025-12-12 12:29:49.145622|
|53336f97ddbe53e2a617d07b17447b9

3096